In [1]:
import robot
import map
import solver
import utils
import scenarios

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

from gtsam.symbol_shorthand import L, X

# unitTest 1: Move robot and check odometrey measurements

In [ ]:
car = robot.robot()

#control inputs
dtheta = np.radians(20)
ds = 0.4

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = utils.spawnWorld(xrange = (-2,2), yrange = (-1,3))
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey((ds,ds,dtheta))
        
        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_GT = np.vstack([hist_GT,car.pose])

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

# unitTest 2: Build prior map

In [ ]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs @ sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [ ]:
#------Build worldmap
xrange = (-2,2); yrange = (-1,3)
fig , ax = utils.spawnWorld(xrange, yrange)
np.random.seed(seed=0)

N = 20 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
car.FOV = 2*np.pi
car.range = 1
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey(odom)

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

        print(meas_lm)


# unitTest4: Plot Robot Cone

In [ ]:
#------Build worldmap
xrange = (-2,2); yrange = (-1,3)
fig , ax = utils.spawnWorld(xrange, yrange)
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

with plt.ion():
    while time <= timeFinal:
        car.moveAndMeasureOdometrey(odom)

        #increase time
        time += dt

        car.plot()
        plt.draw()
        plt.pause(dt)

# unitTest5 GTSAM

notes: </br>
examples folder: /home/alon18/LocalInstalls/miniconda3/envs/smallslam/lib/python3.9/site-packages/gtsam/examples </br>
* imuFactorISAM2Example.py - understand difference between graph.add and graph.push_back </br>
  + never mind, they are the same (?) https://gtsam.org/doxygen/a03480.html#a1bd7154e74538b5934fc616bb5ef7281 </br>
* PlanarSLAMExample.py- factor for range and bearing </br>


from VisualISAMExample.py: </br>
      # If this is the first iteration, add a prior on the first pose to set the coordinate frame </br>
        # and a prior on the first landmark to set the scale </br>

In [2]:
car, worldMap, ax, fig, gt_odom = scenarios.scenario1()
backend = solver.solver(ax = ax, X0cov = car.odometry_noise/1000, semantics = worldMap.exportSemantics())

car.odometry_noise = car.odometry_noise
car.rgbd_noise = car.rgbd_noise
car.FOV = np.pi/2

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_solverResults = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_solverResults.append(backend.current_estimate)

        #plot
        car.plot()
        ax.set_title([lm.index for lm in meas_lms])
        backend.plot_landmarks(plotIndex = True, plotSemantics = True)
        backend.plot_poses()

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        
        plt.draw()
        plt.pause(0.2)

KeyboardInterrupt: 

# Scenario 2 - 4

world2: linear trajectory, robot range = 10, odom_dx = 1, landmarks-yrange = (-2,+2), range = 10 </br>
world3: linear trajectory, robot range = 3, odom_dx = 1, landmarks-yrange = (-2,+2), range = 3 </br>
world4: linear trajectory, robot range = 20, odom_dx = 1, landmarks-yrange = (-10,10), range = 20 </br>

In [2]:
car, worldMap, axWorld, axError,  fig, gt_odom = scenarios.scenario2()
backend = solver.solver(ax = axWorld, X0cov = car.odometry_noise/1000, semantics = worldMap.exportSemantics())

#init history loggers
hist_GT, hist_DR, hist_ISAM2 = car.pose.copy(), car.pose.copy(), car.pose.copy()
hist_DR_error, hist_ISAM2_error = np.zeros((1,1)), np.zeros((1,1))

#set graphics
graphic_GT_traj, = axWorld.plot([], [],'ko-',markersize = 3)
graphic_DR_traj, = axWorld.plot([], [],'ro-',markersize = 3)
graphic_ISAM2_traj, = axWorld.plot([], [],'bo-',markersize = 3)
graphics_DR_error, = axError.plot([], [], 'ro-', markersize = 3)
graphics_ISAM2_error, = axError.plot([], [], 'bo-', markersize = 3)

fig.legend(handles = [graphic_GT_traj,graphic_DR_traj,graphic_ISAM2_traj], 
        labels = ["ground truth","dead reckoning","ISAM2"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])

        xcurrent_ISAM2 = utils.pose2ToNumpy(backend.current_estimate.atPose2(X(backend.i)))

        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_ISAM2 = np.vstack([hist_ISAM2,xcurrent_ISAM2])
        hist_DR_error = np.vstack([hist_DR_error,np.linalg.norm(xcurrent_DR[:2]-car.pose[:2])])
        hist_ISAM2_error = np.vstack([hist_ISAM2_error,np.linalg.norm(xcurrent_ISAM2[:2]-car.pose[:2])])

        #plot
        car.plot()
        axWorld.set_title([lm.index for lm in meas_lms])
        #backend.plot_landmarks(plotIndex = False, plotSemantics = False)
        #backend.plot_poses(plotCov = False)

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        graphic_ISAM2_traj.set_data(hist_ISAM2[:,0],hist_ISAM2[:,1])
        graphics_DR_error.set_data(range(len(hist_DR_error)),hist_DR_error)
        graphics_ISAM2_error.set_data(range(len(hist_ISAM2_error)),hist_ISAM2_error)
        axError.relim()
        axError.autoscale_view(True,True,True)

        plt.show()
        plt.pause(0.1)